## PROMPT
Generate a pipeline for Databricks:
* Get Data information from two web addresses:
   * Space launches: https://api.spacexdata.com/v3/launches
* Filter the list of launches based on launch year and launch success status.
* Send the chosen information to the web address: https://httpbin.org/post
* The script must provide status updates on its progress, report any errors encountered, confirm the outcome of the final data sending step, and measure/report execution times.

In [0]:
import requests
from datetime import datetime
import time

# Step 1: Define helper functions for logging and error handling
def log_message(message):
    """Logs a message with a timestamp."""
    print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {message}")

def measure_execution_time(start_time):
    """Calculates and logs the execution time."""
    elapsed_time = time.time() - start_time
    log_message(f"Execution time: {elapsed_time:.2f} seconds")

# Step 2: Fetch data from the SpaceX API
def fetch_space_launches():
    url = "https://api.spacexdata.com/v3/launches"
    try:
        log_message("Fetching space launch data...")
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        log_message("Data fetched successfully.")
        return response.json()
    except requests.exceptions.RequestException as e:
        log_message(f"Error fetching data: {e}")
        raise

# Step 3: Filter launches based on year and success status
def filter_launches(launches, year, success_status):
    log_message(f"Filtering launches for year {year} with success status {success_status}.")
    filtered_launches = [
        launch for launch in launches
        if launch['launch_year'] == str(year) and launch['launch_success'] == success_status
    ]
    log_message(f"Filtered {len(filtered_launches)} launches.")
    return filtered_launches

# Step 4: Send filtered data to the target web address
def send_data_to_web(filtered_launches):
    url = "https://httpbin.org/post"
    try:
        log_message("Sending filtered data to the web address...")
        response = requests.post(url, json=filtered_launches)
        response.raise_for_status()  # Raise an exception for HTTP errors
        log_message("Data sent successfully.")
        log_message(f"Response from server: {response.json()}")
    except requests.exceptions.RequestException as e:
        log_message(f"Error sending data: {e}")
        raise

# Main pipeline function
def main():
    start_time = time.time()
    
    # Configuration parameters
    target_year = 2020  # Example year to filter
    success_status = True  # Example success status to filter
    
    try:
        # Step 1: Fetch data
        launches = fetch_space_launches()
        
        # Step 2: Filter data
        filtered_launches = filter_launches(launches, target_year, success_status)
        
        # Step 3: Send data
        if filtered_launches:
            send_data_to_web(filtered_launches)
        else:
            log_message("No launches to send after filtering.")
    
    except Exception as e:
        log_message(f"Pipeline failed with error: {e}")
    
    finally:
        # Measure and report execution time
        measure_execution_time(start_time)

# Run the pipeline
if __name__ == "__main__":
    main()